In [ ]:
from pathlib import Path
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import dmidc
import dmidc.harmonie
from mllam_verification.plot import plot_single_metric_gridded_map, plot_single_metric_timeseries, plot_single_metric_hovmoller
from mllam_verification.operations.statistics import rmse
from datetime import datetime

dmidc.__version__

In [ ]:
FORECAST_DURATION = "PT18H"
RELOAD_FC = True

## Load DANRA forecast data

In [ ]:
# Retrieve DANRA forecast data
dir_ = "/home/maf/mllam/data"
if RELOAD_FC:
    ds_t2m_danra_fc = dmidc.harmonie.load(
        suite_name="DANRA",
        analysis_time=slice("2022-01-01", "2022-01-07"),
        data_kind="FORECAST",
        forecast_duration=FORECAST_DURATION,
        short_name=["t"],
        level_type="heightAboveGround",
        level=2,
        temp_filepath=dir_,
    )
    ds_danra_fc = dmidc.harmonie.load(
        suite_name="DANRA",
        analysis_time=slice("2022-01-01", "2022-01-07"),
        data_kind="FORECAST",
        forecast_duration=FORECAST_DURATION,
        short_name=["u", "v"],
        level_type="isobaricInhPa",
        level=[100, 200, 250, 300, 400, 500, 600, 700, 800, 850, 900, 925, 950, 1000],
        temp_filepath=dir_,
    )
else:
    fp_zarr_json = Path(dir_,"analysis_time_20220101-20220107__suite_name_DANRA__data_kind_FC33hr__short_name_t__level_2__level_type_heightAboveGround__forecast_duration_18:00:00.zarr.json")
    ds_t2m_danra_fc  = xr.open_zarr(f"reference::{fp_zarr_json}", consolidated=False)

ds_t2m_danra_fc


## Load DANRA analysis data

In [ ]:
version = "v0.5.0"
fp_root = Path(f"/dmidata/projects/cloudphysics/danra/data/{version}")

ds_danra_analysis_sl = xr.open_zarr(fp_root / "single_levels.zarr")
ds_danra_analysis_sl.attrs['suite_name'] = "danra"

ds_danra_analysis_pl = xr.open_zarr(fp_root / "pressure_levels.zarr")
ds_danra_analysis_pl.attrs['suite_name'] = "danra"

In [ ]:
# Allign coordinates and assign datasource
da_t2m_danra_fc = ds_t2m_danra_fc.assign_coords(
    datasource="DANRA forecast",
    x=ds_danra_analysis_sl.x,
    y=ds_danra_analysis_sl.y,
    lon=ds_danra_analysis_sl.lon,
    lat=ds_danra_analysis_sl.lat,
)

# 2 meter temperature

In [ ]:
# Select the relevant forecast data and adjust the coordinates
parameter = "t"
da_t2m_danra_fc: xr.DataArray = ds_t2m_danra_fc.isel(analysis_time=slice(4))[parameter]
da_t2m_danra_fc


In [ ]:


END_TIME = da_t2m_danra_fc["time"].max().values
da_t2m_danra_analysis = ds_danra_analysis_sl["t2m"].sel(
    time=slice(ds_t2m_danra_fc.analysis_time[0].values + np.timedelta64(3, "h"), END_TIME)
)

### RMSE between DANRA analysis and forecast

In [ ]:
ax = plot_single_metric_timeseries(da_t2m_danra_analysis, da_t2m_danra_fc, time_axis="UTC", stats_operation=rmse, hue="analysis_time")
ax.set_title("RMSE between DANRA forecast and analysis")

### Gridded map of difference between DANRA analysis and forecast

In [ ]:
fig, ax = plt.subplots(figsize=(8, 10), subplot_kw={"projection": ccrs.AlbersEqualArea(central_longitude=11.0, central_latitude=55.0)})

# Select specific analysis time to plot
da_t2m_danra_fc_specific_anal = da_t2m_danra_fc.isel(analysis_time=0)
time_selection = datetime(2022, 1, 1, 18)

ax = plot_single_metric_gridded_map(da_t2m_danra_analysis, da_t2m_danra_fc_specific_anal , time_selection=time_selection, axes=ax, xarray_plot_kwargs={"transform": ccrs.PlateCarree(), "cmap": "RdBu_r"})
ax.coastlines(linewidth=0.5, color="black")
ax.gridlines(draw_labels=["bottom", "left"], color="gray", alpha=0.5, linestyle="--")
ax.set_title("Difference between DANRA forecast and analysis\n at {}".format(time_selection))

In [ ]:
da_t2m_danra_fc_no_nans =  da_t2m_danra_fc.isel(analysis_time=0).dropna(dim="time")
ax = plot_single_metric_hovmoller(da_t2m_danra_analysis, da_t2m_danra_fc_no_nans , preserve_dim="y", stats_operation=rmse, time_axis="UTC", xarray_plot_kwargs={"cmap": "RdBu_r"})